In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
!pip install split-folders
import splitfolders

input_folder = '/content/drive/MyDrive/ig_game_train_set'

output_folder = '/content/'

splitfolders.ratio(input_folder,
                   output=output_folder,
                   seed = 42,
                   ratio = (.7, .0, .3))

train = ImageDataGenerator(rescale= 1./255)
val = ImageDataGenerator(rescale= 1./255)
test = ImageDataGenerator(rescale= 1./255)

train_dataset = train.flow_from_directory("/content/train",
                                          target_size= (150, 150),
                                          batch_size= 32,
                                          class_mode= 'categorical')


test_dataset = test.flow_from_directory("/content/test",
                                          target_size= (150, 150),
                                          batch_size= 32,
                                          class_mode= 'categorical')

Copying files: 4222 files [02:51, 24.63 files/s] 


Found 2952 images belonging to 7 classes.
Found 1270 images belonging to 7 classes.


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
!pip install split-folders
import splitfolders

# 데이터 경로 설정
dataset_path = '/content/train'
test_path = '/content/test'

# 데이터 전처리 및 증강
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2)  # 20%를 검증 데이터로 사용

# 학습 데이터 생성기
train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training')  # 학습 데이터

# 검증 데이터 생성기
validation_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation')  # 검증 데이터

# 클래스 확인
print("Class indices:", train_generator.class_indices)
num_classes = len(train_generator.class_indices)
print(f"Number of classes: {num_classes}")

# 사전 학습된 VGG16 모델 로드 (include_top=False로 최종 레이어 제외)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# 새로운 레이어 추가
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x)

# 모델 정의
model = Model(inputs=base_model.input, outputs=predictions)

# 사전 학습된 레이어 고정
for layer in base_model.layers:
    layer.trainable = False

# 모델 컴파일
model.compile(optimizer=Adam(lr=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 학습률 감소 콜백
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)

# 모델 학습
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=25,
    callbacks=[reduce_lr]
)

# 모델 평가
test_generator = train_datagen.flow_from_directory(
    test_path,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical')

test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f"Test accuracy: {test_acc:.2f}")

# 모델 저장
model.save('game_category_classifier_vgg16.h5')

Found 2365 images belonging to 7 classes.
Found 587 images belonging to 7 classes.
Class indices: {'RPG': 0, '러닝': 1, '시뮬레이션': 2, '전략': 3, '퀴즈': 4, '퍼즐': 5, '하이퍼캐주얼': 6}
Number of classes: 7
58889256/58889256 [==============================] - 3s 0us/step


Epoch 1/25
73/73 [==============================] - 820s 11s/step - loss: 1.8413 - accuracy: 0.2773 - val_loss: 1.6730 - val_accuracy: 0.3663 - lr: 0.0010
Epoch 2/25
73/73 [==============================] - 853s 12s/step - loss: 1.6666 - accuracy: 0.3511 - val_loss: 1.5924 - val_accuracy: 0.3819 - lr: 0.0010
Epoch 3/25
73/73 [==============================] - 800s 11s/step - loss: 1.6159 - accuracy: 0.3639 - val_loss: 1.5652 - val_accuracy: 0.4028 - lr: 0.0010
Epoch 4/25
43/73 [================>.............] - ETA: 4:25 - loss: 1.5624 - accuracy: 0.4108